#### Transform Customer Data
- 1.Remove records with NULL customer_id
- 2.Remove exact duplicate records
- 3.Remove dulicate records based on the created timestamp
- 4.CAST the columns to the correct data type
- 5.write the tranformed data to the silver schema


In [0]:
%sql
select * 
from gizmobox.bronze.py_customers;

In [0]:
df_customers = spark.sql("select * from gizmobox.bronze.py_customers")

In [0]:
display(df_customers)

In [0]:
spark.table("gizmobox.bronze.py_customers").display()

In [0]:
spark.read.table("gizmobox.bronze.py_customers").display()

1.Remove records with NULL customer_id

In [0]:
df_filtered = df_customers.filter('customer_id is not null')

In [0]:
display(df_filtered)

In [0]:
df_filtered = df_customers.filter(df_customers.customer_id.isNotNull())

In [0]:
display(df_filtered)

In [0]:
df_filtered = df_customers.where(df_customers.customer_id.isNotNull())
display(df_filtered)

2.Remove exact duplicate records
- distinct
- drop_duplicates()

In [0]:
df_distnct =df_filtered.distinct()
display(df_distnct)

In [0]:
df_drop_dup = df_filtered.dropDuplicates()
display(df_drop_dup)

3.Remove dulicate records based on the created timestamp

In [0]:
from pyspark.sql import functions as F
d_max_timestamp = (df_distnct.groupBy("customer_id")
                   .agg(F.max("created_timestamp").alias("created_timestamp")))
display(d_max_timestamp)

In [0]:
df_distinct_customer = (
    df_distnct.join(d_max_timestamp,(df_distnct.customer_id == d_max_timestamp.customer_id) & 
                    (df_distnct.created_timestamp == d_max_timestamp.created_timestamp))).select(df_distnct['*'])
display(df_distinct_customer)


4.CAST the columns to the correct data type

In [0]:
df_final = (
    df_distinct_customer.select(df_distinct_customer.created_timestamp.cast('timestamp'),
                                df_distinct_customer.customer_id,
                                df_distinct_customer.customer_name,
                                df_distinct_customer.date_of_birth.cast('date'),
                                df_distinct_customer.email,
                                df_distinct_customer.member_since.cast('date'),
                                df_distinct_customer.telephone)
)
display(df_final)

5.write the tranformed data to the silver schema

In [0]:
df_final.writeTo("gizmobox.silver.py_customers").createOrReplace()

In [0]:
%sql
select * from gizmobox.silver.py_customers;